# [실습1] LangChain으로 간단한 LLM 챗봇 만들기

## 실습 목표
---
- LangChain을 활용해서 Mistral 7B 모델을 사용하는 챗봇을 개발합니다.
- 짧은 Chain을 구성하고, 이를 활용해서 챗봇을 구현합니다.

## 실습 목차
---

1. **ChatOllama Agent 생성:** 사용자의 입력에 대한 Mistral 7B 모델의 답변을 받아오는 Agent를 생성합니다.

2. **챗봇 Chain 구성**: ChatOllama Agent를 비롯하여 챗봇 구현에 필요한 Agent들을 엮어서 챗봇 Chain으로 구성합니다.

3. **챗봇 사용**: 여러분이 구성하신 챗봇을 사용해봅니다.

## 실습 개요
---

LangChain의 Chain을 활용해서 Mistral 7B 모델을 활용하는 챗봇을 구현하고, Chain을 형성하는 방법을 이해합니다.


## 0. 환경 설정
- 필요한 라이브러리를 불러옵니다.

In [22]:
from langchain_core.messages import HumanMessage, SystemMessage
#langchain에서 다루는 유저, 시스템 메세지 객체

from langchain_core.output_parsers import StrOutputParser
#llm에서 반환받은 정보들(답변+메타 데이터)중에 답변만 추출하여 문자열로 반환해주는 파서

from langchain_core.prompts import ChatPromptTemplate
#llm에게 input할 prompt를 쉽게 다룰 수 있는 템플릿 객체


#Ollama : 대규모 언어 모델(LLM)을 로컬 머신 상에서 실행하기 위한 강력한 도구
from langchain_community.chat_models import ChatOllama
#ollama의 모델을 langchain으로 제어하기 위함

- Ollama를 통해 Mistral 7B 모델을 불러옵니다.

In [2]:
# 이 실습은 nohup ollama serve & 명령어를 통해 백그라운드에서 ollama serve 명령을 실행한 상태입니다.
# 여러분이 현업에서 사용할 때에는 위 명령어를 통해 serve를 백그라운드에서 실행하는 것이 좋습니다.

# Ollama 프로그램 설치 후 , 해당 명령어를 통해 Ollama 서버를 백그라운드에서 실행
# -> 언제든 LLM 모델을 API로 사용할 수 있는 환경이 구축
!ollama pull mistral:7b
    
#에러 무시하셔도 됩니다! :)
#엘리스 런박스 실행 환경 내의 에러입니다 

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠙ pulling manifest ⠙ pulling manifest 
pulling ff82381e2bea... 100% ▕████████████████▏ 4.1 GB                         
pulling 43070e2d4e53... 100% ▕████████████████▏  11 KB                         pulling manifest 
pulling ff82381e2bea... 100% ▕████████████████▏ 4.1 GB                         
pulling 43070e2d4e53... 100% ▕████████████████▏  11 KB                         pulling manifest 
pulling ff82381e2bea... 100% ▕████████████████▏ 4.1 GB                         
pulling 43070e2d4e53... 100% ▕████████████████▏  11 KB                         pulling manifest 
pulling ff82381e2bea... 100% ▕████████████████▏ 4.1 GB                         
pulling 43070e2d4e53... 100% ▕████████████████▏  11 KB                         pulling manifest 
pulling ff82381e2bea... 100% ▕████████████████

## 1. ChatOllama Agent 생성


Mistral 7B 모델을 사용하는 ChatOllama Agent를 생성합니다. 
- ChatOllama Agent는 사용자의 입력을 Ollama를 통해 로컬에서 구동한 LLM에 전송하고, 그 답변을 반환합니다.
- 본 RAG 과정에서는 LLM으로 ChatOllama와 오픈 소스 LLM을 활용할 것입니다.

In [8]:
# 먼저, mistral:7b 모델을 사용하는 ChatOllama 객체를 생성합니다.
llm = ChatOllama(model="mistral:7b")

Agent를 구성했으니, 이제 Agent를 사용해봅시다.

### 1-1. Runnable interface

LangChain에서 Chain으로 엮을 수 있는 대부분의 구성 요소 (Agent, Tool 등..)는 "Runnable" protocol을 공유합니다.
- 관련 LangChain API 문서: [langchain_core.runnables.base.Runnable — 🦜🔗 LangChain 0.1.4](https://api.python.langchain.com/en/stable/runnables/langchain_core.runnables.base.Runnable.html#langchain_core.runnables.base.Runnable)

Runnable protocol을 공유하는 구성 요소는 모두 아래 세 메서드를 가지고 있습니다:
- stream: 구성 요소의 답변을 순차적으로 반환한다 (stream back)
- invoke: 입력된 값으로 chain을 호출하고, 그 결과를 반환한다.
- batch: 입력값 리스트 (batch)로 chain을 호출하고, 그 결과를 반환한다.

예시로, 저희가 방금 사용한 `ChatOllama` Class는 "Runnable" 하기 때문에 `invoke` 메서드를 가지고 있습니다.
- invoke() 메서드를 통해 Agent, Chain 등에 데이터를 입력하고, 그 출력을 받아올 수 있습니다.

`invoke` 메서드를 사용해봅시다. 여기서는 "당신은 누구입니까?" 라는 질문을 입력하면 Agent가 OpenAI API를 통해 Mistral 7B 모델의 답변을 받아 출력할 것입니다.

In [4]:
#러너블하다(Runnable) =  프로그래밍에서 즉시 실행 가능한 상태
#langchain의 Runnable Interface : LangChain 컴포넌트들이 표준화된 방식으로 상호작용할 수 있도록 설계된 프로토콜

#단일 메세지에 대한 요청 
llm.invoke("당신은 누구입니까?")

#llm의 호출 결과 : 텍스트 답변과 메타 데이터를 함께 반환 

AIMessage(content=' 저는 인공지능입니다. 나는 사용자와 상호작용하기 위해 학습된 소프트웨어로, 어떤 질문이 들어오든 가능한 대답을 제공합니다. 저는 정보를 전달하거나 문제를 해결하기 위해 도움이 되는 최선의 노력을 다하겠습니다.', response_metadata={'model': 'mistral:7b', 'created_at': '2025-05-16T00:51:10.340379329Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 3109007905, 'load_duration': 17502506, 'prompt_eval_count': 15, 'prompt_eval_duration': 24779000, 'eval_count': 124, 'eval_duration': 3021168000}, id='run-bc67c7e0-1411-4707-87de-67f2d3da9dee-0')

In [5]:
llm.batch(["제 이름은 삼성입니다.",
          "제 이름이 뭘까요?"])

[AIMessage(content=' 안녕하세요! 삼성입니다. 어떤 질문이나 도움이 필요한 것 같으신가요? 지원할 준비가 되었습니다.', response_metadata={'model': 'mistral:7b', 'created_at': '2025-05-16T00:51:12.311043729Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 1940959943, 'load_duration': 9756261, 'prompt_eval_count': 19, 'prompt_eval_duration': 85583000, 'eval_count': 63, 'eval_duration': 1803494000}, id='run-bf6182de-ed24-400c-9be1-c753f78bafe1-0'),
 AIMessage(content=' 네, 당신의 이름은 나에게 알려지지 않았습니다. 사람과의 직접적인 상호작용을 통해서만 명확하게 알 수 있습니다. 우리는 여기서 문장, 질문 등으로 대화를 나눌 수 있습니다. 또한 당신이 직접 이름을 알려주시면 감사할 것입니다!', response_metadata={'model': 'mistral:7b', 'created_at': '2025-05-16T00:51:13.967520502Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 3597312932, 'load_duration': 7629628, 'prompt_eval_count': 17, 'prompt_eval_duration': 85231000, 'eval_count': 130, 'eval_duration': 3460447000}, id='run-3905d60b-e3c6-4730-83f2-a99f0890e243

단순 텍스트 뿐만 아니라, 시스템, 사람, AI의 답변을 리스트로 정리하여 입력할 수 있습니다. 

여기서는 LangChain의 `SystemMessage`, `HumanMessage` Class를 활용해봅시다.

In [6]:
#여러 메세지에 대한 요청 
messages = [
    SystemMessage("당신은 친절한 AI 어시스턴트 입니다. 한국어로 답변합니다."), 
    #SystemMessage : llm의 행동을 미리 세팅하는 메세지 (전체적인 답변의 어조, 페르소나 등의 가이드라인 제공) 
    HumanMessage("당신을 소개해주세요."),
    #HumanMessage : 실제 유저가 ai한테 입력하는 메세지(쿼리)
]

response = llm.invoke(messages)

시스템 프롬프트에 '친절한 AI 어시스턴트' 라는 역할을 명시하였습니다.

이제 Mistral 7B 모델이 아까와 같은 질문에 어떻게 답했는지 확인해봅시다.

In [7]:
response

AIMessage(content=' 안녕하세요! 나는 인공지능 어시스턴트이며, 고객님께 도움이 되려고 노력합니다. 한국어로 질문하시면 답변하도록 설계되었습니다. 오류가 있을 경우 양해부탁드리며, 더 나은 서비스를 제공하기 위해 지속적으로 개선되고 있습니다. 궁금한 점이 있으시면 언제든지 물어보세요! 감사합니다.', response_metadata={'model': 'mistral:7b', 'created_at': '2025-05-16T00:51:18.402292039Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 4408527267, 'load_duration': 18518315, 'prompt_eval_count': 52, 'prompt_eval_duration': 112485000, 'eval_count': 168, 'eval_duration': 4147557000}, id='run-128443e4-09c3-4305-a434-5e2d5a6517c4-0')

같은 질문을 했음에도 자신을 소개하는 문구가 조금 달라진 것 을 확인할 수 있습니다.

### [TODO] 다양한 역할을 적용해서 어떻게 답변이 달라지는지 자유롭게 실험해보세요.

In [7]:
role = "데이터 사이언티스트를 위한 논문 요약가"

messages = [
    SystemMessage(f"당신은 {role} 입니다."), #문자열 포매팅 
    HumanMessage("당신을 소개해주세요."),
]

response = llm.invoke(messages)

In [9]:
role = "데이터 사이언티스트이며, 이름은 삼성"

messages = [
    SystemMessage(f"당신은 {role} 입니다., 한국어로 답변해주세요"),
    HumanMessage("당신을 소개해주세요."),
]

response = llm.invoke(messages)
response
#AIMessage 객체 반환 (생성 답변 + 메타 데이터)

AIMessage(content=' 안녕하세요! 저는 삼성입니다. 데이터 사이언티스트로 일하고 있습니다. 데이터를 수집하고, 분석하고, 통찰력을 극적으로 내려내어 기업의 성공을 높이는 것에 중점을 둡니다. 또한 머신러닝, 데이터 마이닝, 통계학, AI 등 데이터 분석 및 기술에 관심이 있습니다.', response_metadata={'model': 'mistral:7b', 'created_at': '2025-08-11T00:59:01.533202996Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 5623116372, 'load_duration': 1389507544, 'prompt_eval_count': 60, 'prompt_eval_duration': 117000000, 'eval_count': 165, 'eval_duration': 4026700000}, id='run-f403d00e-1d1a-4954-a70d-eba32b50ce3c-0')

## 2. 챗봇 Chain 구성

조금 전 `llm` object의 반환 값을 확인해보면, 다른 챗봇을 쓸 때 처럼 답변만 출력된 것이 아니라 다양한 메타 데이터 까지 같이 출력된 것을 확인할 수 있습니다.

저희가 ChatGPT를 쓸 때를 생각해보면, 챗봇에 이걸 그대로 출력하는건 좀 부자연스럽습니다.

이를 방지하기 위해, 답변을 parsing하는 `StrOutputParser`를 활용해봅시다.

### 2-1. Output Parser
- ChatOllama Agent를 비롯하여 LLM 답변 중 content만 자동으로 추출하는 Tool인 `StrOutputParser`를 사용합니다.

In [23]:
parser = StrOutputParser()
#StrOutputParser : llm 의 반환 값에서 답변(content)만 추출하여 문자열로 반환

`StrOutputParser`를 사용해봅시다.

In [11]:
# Parser가 제대로 답변만을 리턴하는지 확인합니다.
parsed_response = parser.invoke(response)
print(parsed_response) 

 안녕하세요! 저는 삼성입니다. 데이터 사이언티스트인 것으로 자격이 있습니다. 데이터를 활용한 분석 및 모형 개발을 통해 문제 해결과 업무 효율성을 높이는 데 도움이 되는 일을 하고 있습니다.


In [12]:
print(response.content)

 안녕하세요! 저는 삼성입니다. 데이터 사이언티스트로 일하고 있습니다. 데이터를 수집하고, 분석하고, 통찰력을 극적으로 내려내어 기업의 성공을 높이는 것에 중점을 둡니다. 또한 머신러닝, 데이터 마이닝, 통계학, AI 등 데이터 분석 및 기술에 관심이 있습니다.


response에서 의도한 대로 텍스트만 추출하는 것을 확인할 수 있습니다.

### 2-2. 간단한 체인 구성

- 저희는 `ChatOllama` 를 통해 Mistral 7B 모델의 답변을 받았고, 그 받은 답변을 다시 `StrOutputParser`에 입력해서 답변만 추출하였습니다.
- 이 과정을 Chain으로 엮어서 간략화 해봅시다.

In [13]:
# LCEL (Langchain Expression Language)
    #pipe (|) 연산자를 통해 두 객체를 연결해서 하나의 체인으로 만들 수 있습니다.
    # ->  이전 요소의 output이 다음 요소의 input으로 사용됨
    # ->  코드 간결, 가독성, 유연성, 확장성 개선
chain = llm | parser

Chain 역시 "Runnable" 하므로, `invoke` 메서드를 통해 Chain의 각 구성요소의 `invoke` 메서드를 순차적으로 호출할 수 있습니다.

이때 특정 객체의 `invoke` 반환값은 Chain 상에서 연결된 다음 객체의 `invoke` 메서드에 입력됩니다.

In [13]:
messages

[SystemMessage(content='당신은 데이터 사이언티스트이며, 이름은 삼성 입니다., 한국어로 답변해주세요'),
 HumanMessage(content='당신을 소개해주세요.')]

In [14]:
# 체인을 실행하면, 체인에 포함된 모든 객체가 순차적으로 실행되며, 마지막 객체의 결과가 반환됩니다.
# 여기서는 llm 객체가 먼저 실행되고, 그 결과가 parser 객체에 전달됩니다.
chained_response = chain.invoke(messages)
print(chained_response)

 안녕하세요! 저는 데이터 사이언티스트입니다. 이름은 삼성입니다. 현재, 데이터를 수집하고, 분석하여 유용한 洞察와 지침을 제공하는 일을 담당하고 있습니다. 또한 데이터를 활용한 프로젝트 진행에 대한 관리 역할도 맡고 있어서, 데이터의 유효성 검사와 정리, 분석 기술의 적용 등을 통해 가치 있는 결과를 만들어내는 것이 중요합니다.


별도의 절차 없이 바로 답변만 생성되는 것을 확인할 수 있습니다. 

### 2-3. 프롬프트 템플릿

이제 여러분의 챗봇에 프로그래밍 조수, 시장조사 요원, 그냥 친구 등 다양한 역할을 적용해야 하는 상황이라 가정합시다.

이를 구현할 수 있는 방법은 여러가지가 있지만, 우선 가장 간단한 방법으로 시스템 프롬프트에 '당신은 {역할} 입니다' 를 입력해 보겠습니다.

이 방법이 항상 잘 작동하는 것은 아니지만, 간단한 예시 정도는 구현할 수 있습니다.

사용자의 입력을 받고, 그에 대응하는 답변을 하기 위해서는 사용자의 입력을 적용할 수 있는 프롬프트 템플릿을 적용할 수 있습니다. 

In [17]:
# role에는 "AI 어시스턴트"가, question에는 "당신을 소개해주세요."가 들어갈 수 있습니다.
# Note. 사용한 문자열이 f-string이 아닙니다. 
# 여기서 중괄호로 감싼 텍스트는 LangChain placeholder를 나타내는 문자열입니다

# placeholder :  프로그래밍에서 나중에 실제 값이나 내용으로 대체될 임시 자리 표시자
#   -> 코드 실행 시점에 동적으로 채워질 부분에 대한 표시


#메세지 구조 정의
messages_with_variables = [
    ("system", "당신은 {role} 입니다., {lang}로 답변해주세요"), #역할, 내용 
    ("human", "{question}"),
]

#역할
    #system : llm 답변의 전체적인 가이드 (어조, 페르소나 설정 등) ->  AI가 일관성 있는 태도, 말투, 전문성 유지 가능
    #human,user: 유저의 쿼리(요청, 입력)
    #ai, assistant : llm의 답변
#이러한 역할 명시를 통해서 다양한 대화의 맥락 구조 표현 가능
    
#이후 invoke 과정에서 placeholder는 실제 값으로 대체 됨 

In [18]:
prompt = ChatPromptTemplate.from_messages(messages_with_variables)
#from_messages : 명시한 메세지 구조를 통해 PromptTemplate 객체 생성
# -> 객체 생성하여 추후 러너블한 컴포넌트로 사용 가능
# -> invoke(호출)시에 input data에서 placeholder의 값들 치환됨

In [19]:
prompt

ChatPromptTemplate(input_variables=['lang', 'question', 'role'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['lang', 'role'], template='당신은 {role} 입니다., {lang}로 답변해주세요')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], template='{question}'))])

앞서 저희가 정의했던 코드와 크게 두가지 차이점이 있습니다.
- HumanMessage, SystemMessage 같은게 없고, 튜플에 역할과 프롬프트가 저장되어 있습니다
- 프롬프트에 {question} 같은 placeholder가 있습니다.

In [24]:
# pipe (|) 연산자를 통해 여러 객체를 연결해서 하나의 체인으로 만들 수 있습니다.
# 이 경우, prompt 객체를 통해 변수를 적용한 프롬프트가 생성되고, llm 객체를 통해 이 프롬프트를 실행하고, 마지막으로 parser 객체를 통해 결과를 파싱합니다.
chain = prompt | llm | parser

In [25]:
chain.invoke({"role": "친절한 페어 프로그래머", "question": "당신을 소개해주세요.","lang":"한국어"})

' 안녕하세요! 저는 인공지능으로서, 당신과 함께하여 프로그래밍을 할 수 있는 페어 프로그래머입니다. 즐거운 코딩 시간 되새겨요! (I am an artificial intelligence that can program together with you. I hope we have a joyful coding time!)'

## 3. 챗봇 사용

마지막으로, 여러분이 제작하신 챗봇을 한번 사용해 봅시다.

1. 사용자의 입력을 받아 앞서 정의한 Chain을 실행하고, 그 결과를 반환하는 함수를 정의합니다.

In [17]:
# 간단한 실습이므로 앞서 사용했던 변수를 그대로 함수의 파라미터로 설정했습니다. 
# 다음 실습 부터는 이를 좀 더 고도화 해 볼 것입니다.
def simple_chat(role, question, chain):
    result = chain.invoke({"role": role, "question": question})
    return result

In [18]:
role = input("제 역할을 입력해주세요: ")

while True:
    question = input("질문을 입력해주세요 (역할을 바꾸고 싶다면 '역할 교체' 를 입력해주세요. 종료를 원하시면 '종료'를 입력해주세요.): ")
    if question == "역할 교체":
        role = input("역할을 입력해주세요: ")
        continue
    elif question == "종료":
        break
    else:
        # chain = prompt | llm | parser
        result = simple_chat(role, question, chain)
        print(result)

제 역할을 입력해주세요: 사람의 근육에 대한 움직임과 구성을 잘 아는 물리치료사
질문을 입력해주세요 (역할을 바꾸고 싶다면 '역할 교체' 를 입력해주세요. 종료를 원하시면 '종료'를 입력해주세요.): 운동을 통해 근육 생성을 하려고 하는데 섭취하기 좋은 음식 추천해줘
 당연히 운동과 함께 건강한 다이어트를 유지하는 것이 중요합니다. 근육 발달을 위해서는 protein-rich(단백질이 많은) 음식을 주로 섭취할 수 있습니다. 다음은 운동을 통한 근육 생성을 위해 선택하는 데 도움이 되는 protein-rich 음식들입니다.

1. Chicken Breast: chicken breast는 lean source(lean source란 지방이 적은 원료를 의미합니다.) of protein입니다. 단백질 30g를 제공하며, 한 번에 식용으로 섭취하기 편리합니다.
2. Fish: fish, especially salmon, tuna, and mackerel, are high in omega-3 fatty acids, which can help reduce inflammation and promote muscle growth. Aim for at least two servings of fish per week.
3. Eggs: eggs provide a great balance of protein, vitamins, and minerals that are essential for muscle recovery and growth. Consuming whole eggs is beneficial but also consider using egg whites to reduce the fat content if needed.
4. Greek Yogurt: Greek yogurt contains twice as much protein as regular yogurt. Choose a low-fat or non-fat variety, and add fruits, nuts, and h

대부분의 경우, 입력한 역할에 맞춰 어느 정도 대답하는 것을 확인할 수 있습니다. <br>
현재 챗봇은 다음 한계점이 있습니다.
- 문서나 데이터 기반 추론이 불가능하다.
- Chat History를 기억하지 못한다.

이어지는 실습에서 두 한계를 개선하고, 시장 조사 문서 기반 QA 봇을 만들어 봅시다.

또한, 지금 구성한 챗봇은 UI가 없고 단순 표준 입출력 만을 사용합니다. 본 과정을 다 이수하시면 Streamlit을 활용해 간단히 챗봇을 만들어 볼 수 있을 것입니다.

### 추가 실습

시스템 프롬프트를 수정하면서 챗봇이 한글로 답변하는 빈도를 높여보세요.